# Evaluate the performance of the different models by using a sliding window approach

Steps:
1. Load the ground truth signals and harmonize the data
2. Evaluate the performance of the different models by using a sliding window approach

## Step 1: Load the ground truth signals and harmonize the data

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

## Step 2: Evaluate the performance of the different models by using a sliding window approach

In [ ]:
predictions = pd.concat([raft_predictions, pretrained_predictions])
len(predictions)

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import itertools
import scipy.stats as stats
from respiration.analysis import sliding_window_psd

# rPPG lowpass and highpass frequencies
lowpass = 0.7
highpass = 2.5

# Breathing lowpass and highpass frequencies
# lowpass = 0.08
# highpass = 0.5

for model in predictions['model'].unique():
    model_predictions = predictions[predictions['model'] == model]

    gt_psds = []
    pred_psds = []

    subjects = model_predictions['subject'].unique()
    settings = model_predictions['setting'].unique()

    for (subject, setting) in itertools.product(subjects, settings):
        # Get the predicted signal
        pred_signal = model_predictions[
            (model_predictions['subject'] == subject) &
            (model_predictions['setting'] == setting)]['signal'].values[0]

        # Get the ground truth signal
        gt_signal = dataset.get_vital_sign(subject, setting, utils.VitalSigns.pleth)

        prediction_windows = sliding_window_psd(pred_signal, 30, lowpass, highpass, window_size=30)
        gt_psds.extend(prediction_windows)

        gt_windows = sliding_window_psd(gt_signal, 30, lowpass, highpass, window_size=30)
        pred_psds.extend(gt_windows)

    gt_psds = np.array(gt_psds)
    pred_psds = np.array(pred_psds)

    print(f'Model: {model}')
    mse = np.mean((gt_psds - pred_psds) ** 2)
    print(f'MSE: {mse}')

    mae = np.mean(np.abs(gt_psds - pred_psds))
    print(f'MAE: {mae}')

    corr, p = stats.pearsonr(gt_psds.flatten(), pred_psds.flatten())
    print(f'Correlation: {corr}, p-value: {p}')
    print()
    #break